CNN으로 MNIST 분류하기

우선 필요한 도구들을 임포트합니다.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 됩니다.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

학습에 사용할 파라미터를 설정합니다.

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

데이터로더를 사용하여 데이터를 다루기 위해서 데이터셋을 정의해줍니다.

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True, #True는 train 데이터
                          transform=transforms.ToTensor(), #데이터를 PyTorch 텐서로 변환
                          download=True) #해당 경로에 MNIST 데이터가 없다면 다운로드

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train = False, #False는 test 데이터
                         transform=transforms.ToTensor(), #데이터를 PyTorch 텐서로 변환
                         download=True) #해당 경로에 MNIST 데이터가 없다면 다운로드

0.0%

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

91.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, # mnist_train 데이터를 로드할 대상으로 정해줍니다.
                                          batch_size = batch_size, # 위에서 정의한 배치크기를 사용합니다.
                                          shuffle = True, # Epoch마다 데이터 셋을 섞어서 데이터가 학습되는 순서를 바꾸겠습니다.
                                          drop_last = True) #마지막 배치를 버리겠습니다.

이제 클래스로 모델을 설계합니다.

In [11]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), # 1번 레이어 : 합성곱층(Convolutional layer)
            # 합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1)
            torch.nn.ReLU(),# 활성화 함수 ReLU
            torch.nn.MaxPool2d(kernel_size=2, stride=2))# 맥스풀링(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            # 2번 레이어 : 합성곱층(Convolutional layer)
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            # 합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1)
            torch.nn.ReLU(),# 활성화 함수 ReLU
            torch.nn.MaxPool2d(kernel_size=2, stride=2))# 맥스풀링(kernel_size=2, stride=2))
        
        # fc layer 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        # fc layer 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(-1,7*7*64)
        out = self.fc(out)
        return out

모델을 정의합니다.

In [12]:
# instantiate CNN model
model = CNN().to(device)

비용 함수와 옵티마이저를 정의합니다.

In [13]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) # CrossEntropyLoss()를 사용합니다.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # Adam optimizer를 사용하고 위에서 정의한 learning rate를 사용합니다.

총 배치의 수를 출력해보겠습니다.

In [14]:
total_batch = len(data_loader)
print(total_batch)

600


총 배치의 수는 600입니다. 그런데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개란 의미입니다. 이제 모델을 훈련시켜보겠습니다. (시간이 꽤 오래 걸립니다.)

In [16]:
# train my model

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad() # gradient를 0으로 초기화
        hypothesis = model(X) #모델을 생성해서 hypothesis를 구해보세요.
        cost = criterion(hypothesis, Y) #위에서 정의한 비용함수를 사용하세요.
        cost.backward() # 비용 함수를 미분하여 gradient 계산
        optimizer.step() # W와 b를 업데이트

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 0.22688581
[Epoch:    2] cost = 0.0599546693
[Epoch:    3] cost = 0.0435765237
[Epoch:    4] cost = 0.0345481746
[Epoch:    5] cost = 0.0292061958
[Epoch:    6] cost = 0.0235030316
[Epoch:    7] cost = 0.0211836938
[Epoch:    8] cost = 0.0171417687
[Epoch:    9] cost = 0.0139503237
[Epoch:   10] cost = 0.0124029312
[Epoch:   11] cost = 0.0102563761
[Epoch:   12] cost = 0.00917423237
[Epoch:   13] cost = 0.00739759579
[Epoch:   14] cost = 0.00699632801
[Epoch:   15] cost = 0.00555367861
Learning Finished!


이제 테스트를 해보겠습니다.

학습을 진행하지 않을 것이므로 torch.no_grad()

In [17]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9860000014305115


98%의 정확도를 얻습니다. 다음 챕터에서는 층을 더 쌓아보겠습니다.